In [6]:
from pydantic_ai import Agent
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Simple Hello World Example with just a model
agent = Agent(
    'gemini-1.5-flash',
    system_prompt='be concise, reply with only one setence',
)

result = agent.run_sync('Where does "hello world" come from?')
print(result.data)

The phrase "hello, world" originates from Brian Kernighan's 1972 tutorial "A Tutorial Introduction to the Language B".



In [ ]:
# Not very interesting yet, 
# but we can easily add "tools", dynamic system prompts, and structured responses to build more powerful agents.

from dataclasses import dataclass

from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext

from bank_support import DatabaseConn  ## this is a fake db connection in bank_support.py


In [ ]:
@dataclass
class SupportDependencies:
    customer_id: int
    db: DatabaseConn

class SupportResult(BaseModel):
    support_advice: str = Field(description='Advice returned to the customer')
    block_card: bool = Field(description="Where to block the customer's card")
    risk: int = Field(description='Risk level of the query', ge=0, le=10)

### Create the Agent!
support_agent = Agent(
    'gemini-1.5-flash',
    deps_type=SupportDependencies,  # the agent dependency required inputs
    result_type=SupportResult,      # the structured output of the agent
    system_prompt= (
        'You are a support agent in our bank, give the customer support '
        'and judge the risk level of their query.'
    ),
)

# Add system prompt and tools available to the support_agent
@support_agent.system_prompt
async def add_customer_name(ctx: RunContext[SupportDependencies]) -> str:
    customer_name = await ctx.deps.db.customer_name(id=ctx.deps.customer_id)